# Core 

> Minimal pipeline for Diffusion Guidance experiments.

In [ ]:
#| default_exp guidance

In [ ]:
#| export
from dynamic_cfg.kdiff import *
from dynamic_cfg.schedules import name2schedule
from dynamic_cfg.normalizers import name2norm

In [ ]:
#| export

class DynamicCFG:
    def __init__(self, norm_name='no_norm', schedule_name='constant'):
        self.normalizer = name2norm[norm_name]()
        self.scheduler = name2schedule[schedule_name]()


    def guide(self, uncond, cond, ts):
        """Applies dynamic Classifier-free Guidance.
        """
        # set the conditional and unconditional vectors
        self.normalizer.set_latents(u=uncond, t=cond)
        # compute the guidance update vector: (cond - uncond)
        self.normalizer.compute_update()

        # apply optional pre-processing
        self.normalizer.pre_proc()

        # get the current, scheduled guidance scale 
        guidance_scale = self.scheduler.value_of(param='g', at=ts)
        # run classifier-free guidance 
        self.normalizer.apply_cfg(guidance_scale)

        # apply optional post-processing
        self.normalizer.post_proc()

        # return the dynamic noise prediction
        return self.normalizer.get_pred()


    def update_sched_kwargs(self, other_kwargs):
        self.scheduler.update_sched_kwargs(other_kwargs)


    def set_timesteps(self, num_steps):
        self.scheduler.set_num_steps(num_steps)
        self.scheduler.set_guidance_schedule()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()